# Defeat learners
ML for trading Udacity Course exercise

More info:
https://quantsoftware.gatech.edu/Defeat_learners

A transcription of the Udacity Course lectures can be find on https://docs.google.com/document/d/1ELqlnuTSdc9-MDHOkV0uvSY4RmI1eslyQlU9DgOY_jc/edit?usp=sharing

Kairosart 2018



## Overview

In this project you will generate data that you believe will work better for one learner than another. This will test your understanding of the strengths and weaknesses of various learners. The two learners you should aim your datasets at are:

    * A decision tree learner with leaf_size = 1 (DTLearner). Note that for testing purposes we will use our implementation of DTLearner
    * A LinRegLearner provided as part of the repo.

Your data generation should use a random number generator as part of its data generation process. We will pass your generators a random number seed. Whenever the seed is the same you should return exactly the same data set. Different seeds should result in different data sets. 

## Import libraries

In [88]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from copy import deepcopy
from collections import Counter
from operator import itemgetter
import math  

##  best4LinReg Function

In [23]:
def best4LinReg(seed=np.random.randint(1000000)):
    """This function should return a dataset (X and Y) that will work
    better for linear regression than decision trees
    Parameters: 
    seed: Random integer used to initialize the pseudo-random number generator. 
    Whenever the seed is the same, the same data set will be returned. 
    Different seeds should result in different data sets.
    Returns: 
    X: A numpy ndarray of X values
    Y: A numpy 1D array of Y values
    """

    np.random.seed(seed)

    # X and Y should each contain from 10 to 1000 rows
    num_rows = np.random.randint(10, 1001)

    # X should have from 2 to 10 columns
    num_X_cols = np.random.randint(2, 11)

    X = np.random.normal(size=(num_rows, num_X_cols))
    Y = np.zeros(num_rows)
    for col in range(num_X_cols):
        Y += X[:, col]

    return X, Y

## best4DT Function

In [23]:
def best4DT(seed=np.random.randint(1000000)):
    """This function should return a dataset (X and Y) that will work
    better for decision trees than linear regression
    Parameters: 
    seed: Random seed used to initialize the pseudo-random number generator. 
    Whenever the seed is the same, the same data set will be returned. 
    Different seeds should result in different data sets.
    Returns: 
    X: A numpy ndarray of X values
    Y: A numpy 1D array of Y values
    """

    np.random.seed(seed)
    
    # X and Y should each contain from 10 to 1000 rows
    num_rows = np.random.randint(10, 1001)

    # X should have from 2 to 10 columns
    num_X_cols = np.random.randint(2, 11)

    X = np.random.normal(size=(num_rows, num_X_cols))
    Y = np.zeros(num_rows)
    for col in range(num_X_cols):
        Y += X[:, col] ** 2
    
    return X, Y

## Generate data to fool learners

In [ ]:
X1, Y1 = best4LinReg(seed = 5)
X2, Y2 = best4DT(seed = 5)


### Deductions:

    * Does either dataset returned contain fewer or more than the allowed number of samples?
    * Does either dataset returned contain fewer or more than the allowed number of dimensions in X?
    * When the seed is the same does the best4LinReg dataset generator return the same data?
    * When the seed is the same does the best4DT dataset generator return the same data?
    * When the seed is different does the best4LinReg dataset generator return different data?
    * When the seed is different does the best4DT dataset generator return different data?
    * Does the code attempt to import a learner? 

In [27]:
# Number of samples
print("Samples: ", X1.shape[0], X2.shape[0]) 

# Dimensions
print("Dimensions: ", X1.shape, X2.shape)

# best4LinReg dataset generator
print("When the seed is the same does the best4LinReg dataset generator return the same data.")

# best4DT dataset generator
print("When the seed is the same does the best4DT dataset generator return the same data.")

Samples:  877 877
Dimensions:  (877, 8) (877, 8)
When the seed is the same does the best4LinReg dataset generator return the same data.
When the seed is the same does the best4DT dataset generator return the same data.


## DTLearner

A simple wrapper for Decision Tree regression.

In [79]:
class DTLearner(object):

    def __init__(self, leaf_size=1, verbose=False, tree=None):
        self.leaf_size = leaf_size
        self.verbose = verbose
        self.tree = deepcopy(tree)
        if verbose:
            self.get_learner_info()
        

    def __build_tree(self, dataX, dataY, rootX=[], rootY=[]):
        """Builds the Decision Tree recursively by choosing the best feature to split on and 
        the splitting value. The best feature has the highest absolute correlation with dataY. 
        If all features have the same absolute correlation, choose the first feature. The 
        splitting value is the median of the data according to the best feature. 
        If the best feature doesn't split the data into two groups, choose the second best 
        one and so on; if none of the features does, return leaf
        Parameters:
        dataX: A numpy ndarray of X values at each node
        dataY: A numpy 1D array of Y values at each node
        rootX: A numpy ndarray of X values at the parent/root node of the current one
        rootY: A numpy 1D array of Y values at the parent/root node of the current one
        
        Returns:
        tree: A numpy ndarray. Each row represents a node and four columns are feature indices 
        (int type; index for a leaf is -1), splitting values, and starting rows, from the current 
        root, for its left and right subtrees (if any)
        """
        # Get the number of samples (rows) and features (columns) of dataX
        num_samples = dataX.shape[0]
        num_feats = dataX.shape[1]
        
        
        # If there is no sample left, return the most common value from the root of current node
        if num_samples == 0:
            return np.array([-1, Counter(rootY).most_common(1)[0][0], np.nan, np.nan])

        # If there are <= leaf_size samples or all data in dataY are the same, return leaf
        if num_samples <= self.leaf_size or len(pd.unique(dataY)) == 1:
            return np.array([-1, Counter(dataY).most_common(1)[0][0], np.nan, np.nan])
    
        avail_feats_for_split = list(range(num_feats))

        # Get a list of tuples of features and their correlations with dataY
        feats_corrs = []
        for feat_i in range(num_feats):
            abs_corr = abs(pearsonr(dataX[:, feat_i], dataY)[0])
            feats_corrs.append((feat_i, abs_corr))
        
        # Sort the list in descending order by correlation
        feats_corrs = sorted(feats_corrs, key=itemgetter(1), reverse=True)

        # Choose the best feature, if any, by iterating over feats_corrs
        feat_corr_i = 0
        while len(avail_feats_for_split) > 0:
            best_feat_i = feats_corrs[feat_corr_i][0]
            best_abs_corr = feats_corrs[feat_corr_i][1]

            # Split the data according to the best feature
            split_val = np.median(dataX[:, best_feat_i])

            # Logical arrays for indexing
            left_index = dataX[:, best_feat_i] <= split_val
            right_index = dataX[:, best_feat_i] > split_val

            # If we can split the data into two groups, then break out of the loop            
            if len(np.unique(left_index)) != 1:
                break
            
            avail_feats_for_split.remove(best_feat_i)
            feat_corr_i += 1
        
        # If we complete the while loop and run out of features to split, return leaf
        if len(avail_feats_for_split) == 0:
            return np.array([-1, Counter(dataY).most_common(1)[0][0], np.nan, np.nan])

        # Build left and right branches and the root                    
        lefttree = self.__build_tree(dataX[left_index], dataY[left_index], dataX, dataY)
        righttree = self.__build_tree(dataX[right_index], dataY[right_index], dataX, dataY)

        # Set the starting row for the right subtree of the current root
        if lefttree.ndim == 1:
            righttree_start = 2 # The right subtree starts 2 rows down
        elif lefttree.ndim > 1:
            righttree_start = lefttree.shape[0] + 1
        root = np.array([best_feat_i, split_val, 1, righttree_start])

        return np.vstack((root, lefttree, righttree))
    

    def __tree_search(self, point, row):
        """A private function to be used with query. It recursively searches 
        the decision tree matrix and returns a predicted value for point
        Parameters:
        point: A numpy 1D array of test query
        row: The row of the decision tree matrix to search
    
        Returns 
        pred: The predicted value
        """

        # Get the feature on the row and its corresponding splitting value
        feat, split_val = self.tree[row, 0:2]
        
        # If splitting value of feature is -1, we have reached a leaf so return it
        if feat == -1:
            return split_val

        # If the corresponding feature's value from point <= split_val, go to the left tree
        elif point[int(feat)] <= split_val:
            pred = self.__tree_search(point, row + int(self.tree[row, 2]))

        # Otherwise, go to the right tree
        else:
            pred = self.__tree_search(point, row + int(self.tree[row, 3]))
        
        return pred


    def addEvidence(self, dataX, dataY):
        """Add training data to learner
        Parameters:
        dataX: A numpy ndarray of X values of data to add
        dataY: A numpy 1D array of Y training values
        Returns: An updated tree matrix for DTLearner
        """

        new_tree = self.__build_tree(dataX, dataY)

        # If self.tree is currently None, simply assign new_tree to it
        if self.tree is None:
            self.tree = new_tree

        # Otherwise, append new_tree to self.tree
        else:
            self.tree = np.vstack((self.tree, new_tree))
        
        # If there is only a single row, expand tree to a numpy ndarray for consistency
        if len(self.tree.shape) == 1:
            self.tree = np.expand_dims(self.tree, axis=0)
        
        if self.verbose:
            self.get_learner_info()
        
        
    def query(self, points):
        """Estimates a set of test points given the model we built
        
        Parameters:
        points: A numpy ndarray of test queries
        Returns: 
        preds: A numpy 1D array of the estimated values
        """

        preds = []
        for point in points:
            preds.append(self.__tree_search(point, row=0))
        return np.asarray(preds)


    def get_learner_info(self):
        print ("Info about this Decision Tree Learner:")
        print ("leaf_size =", self.leaf_size)
        if self.tree is not None:
            print ("tree shape =", self.tree.shape)
            print ("tree as a matrix:")
            # Create a dataframe from tree for a user-friendly view
            df_tree = pd.DataFrame(self.tree, columns=["factor", "split_val", "left", "right"])
            df_tree.index.name = "node"
            print (df_tree)
        else:
            print ("Tree has no data")

## Some data to test the DTLearner

In [80]:
print ("This is a Decision Tree Learner\n")

# Some data to test the DTLearner
x0 = np.array([0.885, 0.725, 0.560, 0.735, 0.610, 0.260, 0.500, 0.320])
x1 = np.array([0.330, 0.390, 0.500, 0.570, 0.630, 0.630, 0.680, 0.780])
x2 = np.array([9.100, 10.900, 9.400, 9.800, 8.400, 11.800, 10.500, 10.000])
x = np.array([x0, x1, x2]).T

y = np.array([4.000, 5.000, 6.000, 5.000, 3.000, 8.000, 7.000, 6.000])


# Create a tree learner from given train X and y
dtl = DTLearner(verbose=True, leaf_size=1)
print ("\nAdd data")
dtl.addEvidence(x, y)

print ("\nCreate another tree learner from an existing tree")
dtl2 = DTLearner(tree=dtl.tree)

# dtl2 should have the same tree as dtl
assert np.any(dtl.tree == dtl2.tree)

dtl2.get_learner_info()

# Modify the dtl2.tree and assert that this doesn't affect dtl.tree
dtl2.tree[0] = np.arange(dtl2.tree.shape[1])
assert np.any(dtl.tree != dtl2.tree)

# Query with dummy data
dtl.query(np.array([[1, 2, 3], [0.2, 12, 12]]))

# Another dataset to test that "If the best feature doesn't split the data into two
# groups, choose the second best one and so on; if none of the features does, return leaf"
x2 = np.array([
 [  0.26,    0.63,   11.8  ],
 [  0.26,    0.63,   11.8  ],
 [  0.32,    0.78,   10.   ],
 [  0.32,    0.78,   10.   ],
 [  0.32,    0.78,   10.   ],
 [  0.735,   0.57,    9.8  ],
 [  0.26,    0.63,   11.8  ],
 [  0.61,    0.63,    8.4  ]])

y2 = np.array([ 8.,  8.,  6.,  6.,  6.,  5.,  8.,  3.])

dtl = DTLearner(verbose=True)
dtl.addEvidence(x2, y2)

This is a Decision Tree Learner

Info about this Decision Tree Learner:
leaf_size = 1
Tree has no data

Add data
Info about this Decision Tree Learner:
leaf_size = 1
tree shape = (15, 4)
tree as a matrix:
      factor  split_val  left  right
node                                
0        2.0     9.9000   1.0    8.0
1        2.0     9.2500   1.0    4.0
2        0.0     0.7475   1.0    2.0
3       -1.0     3.0000   NaN    NaN
4       -1.0     4.0000   NaN    NaN
5        0.0     0.6475   1.0    2.0
6       -1.0     6.0000   NaN    NaN
7       -1.0     5.0000   NaN    NaN
8        0.0     0.4100   1.0    4.0
9        0.0     0.2900   1.0    2.0
10      -1.0     8.0000   NaN    NaN
11      -1.0     6.0000   NaN    NaN
12       0.0     0.6125   1.0    2.0
13      -1.0     7.0000   NaN    NaN
14      -1.0     5.0000   NaN    NaN

Create another tree learner from an existing tree
Info about this Decision Tree Learner:
leaf_size = 1
tree shape = (15, 4)
tree as a matrix:
      factor  split_val

## LinRegLearner

Linear Regression Learner

In [291]:
class LinRegLearner(object):  
    
    def __init__(self, verbose = False):
        pass
    
    def addEvidence(self,dataX,dataY):  
        """ 
        @summary: Add training data to learner  
        @param dataX: X values of data to add  
        @param dataY: the Y training values  
        """  
        # slap on 1s column so linear regression finds a constant term 
        newdataX = np.ones([dataX.shape[0],dataX.shape[1]+1])  
        newdataX[:,0:dataX.shape[1]]=dataX  
 
        # build and save the model  
        self.model_coefs, residuals, rank, s = np.linalg.lstsq(newdataX, dataY)  
 
    def query(self,points): 
        """  
        @summary: Estimate a set of test points given the model we built.  
        @param points: should be a numpy array with each row corresponding to a specific query.  
        @returns the estimated values according to the saved model.  
        """  
        return (self.model_coefs[:-1] * points).sum(axis = 1) + self.model_coefs[-1]  

## Compare two learners' rmse out of sample

In [292]:
def compare_os_rmse(learner1, learner2, X, Y):  
    # compute how much of the data is training and testing   
    train_rows = int(math.floor(0.6* X.shape[0])) 
    test_rows = X.shape[0] - train_rows     
 
    # separate out training and testing data  
    train = np.random.choice(X.shape[0], size=train_rows, replace=False) 
    test = np.setdiff1d(np.array(range(X.shape[0])), train)  
    trainX = X[train, :]
    trainY = Y[train]  
    testX = X[test, :]  
    testY = Y[test] 
    
    # train the learners 
    learner1.addEvidence(trainX, trainY) # train it  
    learner2.addEvidence(trainX, trainY) # train it 

    # evaluate learner1 out of sample  
    predY = learner1.query(testX) # get the predictions 
    rmse1 = math.sqrt(((testY - predY) ** 2).sum()/testY.shape[0])    

    # evaluate learner2 out of sample 
    predY = learner2.query(testX) # get the predictions 
    rmse2 = math.sqrt(((testY - predY) ** 2).sum()/testY.shape[0])
    return rmse1, rmse2 

def test_code(): 
    # create two learners and get data  
    lrlearner = LinRegLearner(verbose = False)   
    dtlearner = DTLearner(verbose = False, leaf_size = 1)   
    X, Y = best4LinReg()  

    # compare the two learners  
    rmseLR, rmseDT = compare_os_rmse(lrlearner, dtlearner, X, Y)  
    print("rmseLR, rmseDT:", rmseLR, rmseDT)
    # share results    
    print 
    print("best4LinReg() results")
    print("RMSE LR    : ", rmseLR)  
    print("RMSE DT    : ", rmseDT)
    if rmseLR < 0.9 * rmseDT:  
        print("LR < 0.9 DT:  pass")
    else:  
        print("LR >= 0.9 DT:  fail")  
    print  
  
    # get data that is best for a random tree  
    lrlearner = LinRegLearner(verbose = False) 
    dtlearner = DTLearner(verbose = False, leaf_size = 1) 
    X, Y = best4DT()  

    # compare the two learners  
    rmseLR, rmseDT = compare_os_rmse(lrlearner, dtlearner, X, Y)  

    # share results  
    print  
    print("best4RT() results")
    print("RMSE LR    : ", rmseLR)  
    print("RMSE RT    : ", rmseDT)  
    if rmseDT < 0.9 * rmseLR:  
        print("DT < 0.9 LR:  pass")  
    else:    
        print("DT >= 0.9 LR:  fail") 
    print

## Learner test
Code that calls the two data set generating functions and tests them against the two learners.

In [293]:
test_code()

/home/emi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
/home/emi/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3013: RuntimeWarning: invalid value encountered in double_scalars
  prob = _betai(0.5*df, 0.5, df/(df+t_squared))


rmseLR, rmseDT: 1.4681229241792225e-15 0.6379561784404958
best4LinReg() results
RMSE LR    :  1.4681229241792225e-15
RMSE DT    :  0.6379561784404958
LR < 0.9 DT:  pass
best4RT() results
RMSE LR    :  4.1262487946016675
RMSE RT    :  5.017815451510112
DT >= 0.9 LR:  fail


## Grade best4
The grading script.

For best4LinReg (1 test case):

    * We will call best4LinReg 15 times, and select the 10 best datasets. For each successful test +5 points (total of 50 points)
    * For each test case we will randomly select 60% of the data for training and 40% for testing.
    * Success for each case is defined as: RMSE LinReg < RMSE DT * 0.9

For best4DT (1 test case):

    * We will call best4DT 15 times, and select the 10 best datasets. For each successful test +5 points (total of 50 points)
    * For each test case we will randomly select 60% of the data for training and 40% for testing.
    * Success for each case is defined as: RMSE DT < RMSE LinReg * 0.9

In [297]:
# Add parent directory PATH for looking for modules,
import sys
sys.path.insert(0,'..')

import grading
from grading.grading import Grader
import pytest
import os

import traceback as tb
from collections import namedtuple
import time
from functools import cmp_to_key

seconds_per_test_case = 5

max_points = 100.0 
html_pre_block = True  # surround comments with HTML <pre> tag (for T-Square comments field)

# Test cases
Best4TestCase = namedtuple('Best4TestCase', ['description', 'group','max_tests','needed_wins','row_limits','col_limits','seed'])
best4_test_cases = [
    Best4TestCase(
        description="Test Case 1: Best4LinReg",
        group="best4lr",
        max_tests=15,
        needed_wins=10,
        row_limits=(10,1001),
        col_limits=(2,11),
        seed=1489683274
        ),
    Best4TestCase(
        description="Test Case 2: Best4DT",
        group="best4dt",
        max_tests=15,
        needed_wins=10,
        row_limits=(10,1001),
        col_limits=(2,11),
        seed=1489683274
        )]

def keyfunction(a, b):
    return int((b[0]-b[1])-(a[0]-a[1]))


def keyfunction_np_sign(a, b):
    return int(np.sign((b[0]-b[1])-(a[0]-a[1])))

# Test functon(s)
@pytest.mark.parametrize("description,group,max_tests,needed_wins,row_limits,col_limits,seed", best4_test_cases)
def test_learners(description, group, max_tests, needed_wins, row_limits, col_limits, seed, grader):
    """Test data generation methods beat given learner.
    Requires test description, test case group, and a grader fixture.
    """

    points_earned = 0.0  # initialize points for this test case
    incorrect = True
    msgs = []
    try:
        dataX, dataY = None,None
        same_dataX, same_dataY = None,None
        diff_dataX, diff_dataY = None,None
        betterLearner, worseLearner = None, None
        
        if group=="best4dt":
            
            dataX, dataY = run_with_timeout(best4DT,seconds_per_test_case,(),{'seed':seed})
            same_dataX,same_dataY = run_with_timeout(best4DT,seconds_per_test_case,(),{'seed':seed})
            diff_dataX,diff_dataY = run_with_timeout(best4DT,seconds_per_test_case,(),{'seed':seed+1})
            betterLearner = DTLearner
            worseLearner = LinRegLearner
        elif group=='best4lr':
            
            dataX, dataY = run_with_timeout(best4LinReg,seconds_per_test_case,(),{'seed':seed})
            same_dataX, same_dataY = run_with_timeout(best4LinReg,seconds_per_test_case,(),{'seed':seed})
            diff_dataX, diff_dataY = run_with_timeout(best4LinReg,seconds_per_test_case,(),{'seed':seed+1})
            betterLearner = LinRegLearner
            worseLearner = DTLearner

        num_samples = dataX.shape[0]
        cutoff = int(num_samples*0.6)
        worse_better_err = []
        for run in range(max_tests):
            permutation = np.random.permutation(num_samples)
            train_X,train_Y = dataX[permutation[:cutoff]], dataY[permutation[:cutoff]]
            test_X,test_Y = dataX[permutation[cutoff:]], dataY[permutation[cutoff:]]
            better = betterLearner()
            worse = worseLearner()
            better.addEvidence(train_X,train_Y)
            worse.addEvidence(train_X,train_Y)
            better_pred = better.query(test_X)
            worse_pred = worse.query(test_X)
            better_err = np.linalg.norm(test_Y-better_pred)
            worse_err = np.linalg.norm(test_Y-worse_pred)
            worse_better_err.append( (worse_err,better_err) )
        
        worse_better_err.sort(key=cmp_to_key(keyfunction))
        better_wins_count = 0
        for worse_err,better_err in worse_better_err: 
            if better_err < 0.9*worse_err:
                better_wins_count = better_wins_count+1
                points_earned += 5.0
            if better_wins_count >= needed_wins:
                break
        incorrect = False
        if (dataX.shape[0] < row_limits[0]) or (dataX.shape[0]>row_limits[1]):
            incorrect = True
            msgs.append("    Invalid number of rows. Should be between {}, found {}".format(row_limits,dataX.shape[0]))
            points_earned = max(0,points_earned-20)
        if (dataX.shape[1] < col_limits[0]) or (dataX.shape[1]>col_limits[1]):
            incorrect = True
            msgs.append("    Invalid number of columns. Should be between {}, found {}".format(col_limits,dataX.shape[1]))
            points_earned = max(0,points_earned-20)
        if better_wins_count < needed_wins:
            incorrect = True
            msgs.append("    Better learner did not exceed worse learner. Expected {}, found {}".format(needed_wins,better_wins_count))
        if not(np.array_equal(same_dataY,dataY)) or not(np.array_equal(same_dataX,dataX)):
            incorrect = True
            msgs.append("    Did not produce the same data with the same seed.\n"+\
                        "      First dataX:\n{}\n".format(dataX)+\
                        "      Second dataX:\n{}\n".format(same_dataX)+\
                        "      First dataY:\n{}\n".format(dataY)+\
                        "      Second dataY:\n{}\n".format(same_dataY))
            points_earned = max(0,points_earned-20)
        if np.array_equal(diff_dataY,dataY) and np.array_equal(diff_dataX,dataX):
            incorrect = True
            msgs.append("    Did not produce different data with different seeds.\n"+\
                        "      First dataX:\n{}\n".format(dataX)+\
                        "      Second dataX:\n{}\n".format(diff_dataX)+\
                        "      First dataY:\n{}\n".format(dataY)+\
                        "      Second dataY:\n{}\n".format(diff_dataY))
            points_earned = max(0,points_earned-20)  
        
        p1 = Grader()
        if incorrect:
            if group=='author':
                raise (IncorrectOutput)
            else:
                inputs_str = "    Residuals: {0}".format(worse_better_err)
                raise (Exception("Test failed on one or more output criteria.\n  Inputs:\n{0}\n  Failures:\n{1}".format(inputs_str, "\n".join(msgs))))
        else:
            if group != 'author':
                avg_ratio = 0.0
                worse_better_err.sort(key=cmp_to_key(keyfunction_np_sign))
                
                for we,be in worse_better_err[:10]:
                    avg_ratio += (float(we) - float(be))
                   
                avg_ratio = avg_ratio/10.0
                if group=="best4dt":
                    p1 = Grader(np.array([avg_ratio,0]))
                    p1.add_performance(p1)
                else:
                    p1 = Grader(np.array([0,avg_ratio]))
                    p1.add_performance(p1)
    except Exception as e:
        # Test result: failed
        msg = "Description: {} (group: {})\n".format(description, group)
        
        # Generate a filtered stacktrace, only showing erroneous lines in student file(s)
        tb_list = tb.extract_tb(sys.exc_info()[2])
        for i in range(len(tb_list)):
            row = tb_list[i]
            tb_list[i] = (os.path.basename(row[0]), row[1], row[2], row[3])  # show only filename instead of long absolute path
        tb_list = [row for row in tb_list if (row[0] == 'gen_data.py')]
        if tb_list:
            msg += "Traceback:\n"
            msg += ''.join(tb.format_list(tb_list))  # contains newlines
        elif 'grading_traceback' in dir(e):
            msg += "Traceback:\n"
            msg += ''.join(tb.format_list(e.grading_traceback))
        msg += "{}: {}".format(e.__class__.__name__, str(e))

        # Report failure result to grader, with stacktrace
        p1.add_result(GradeResult(outcome='failed', points=points_earned, msg=msg))
        raise
    else:
        # Test result: passed (no exceptions)
        p1.add_result(GradeResult(outcome='passed', points=points_earned, msg=None))

    
    
    return points_earned


In [295]:
# Test Case 1: Best4LinReg Total Points
tp1 = test_learners(*best4_test_cases[0], best4_test_cases)
print("Test Case 1: Best4LinReg Total points: ", tp1)

/home/emi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
/home/emi/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3013: RuntimeWarning: invalid value encountered in double_scalars
  prob = _betai(0.5*df, 0.5, df/(df+t_squared))


Test Case 1: Best4LinReg Total points:  50.0


In [298]:
# Test Case 2: Best4LinReg Total Points
tp2 = test_learners(*best4_test_cases[1], best4_test_cases)
print("Test Case 2: Best4DT Total points: ", tp2)

/home/emi/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:3013: RuntimeWarning: invalid value encountered in double_scalars
  prob = _betai(0.5*df, 0.5, df/(df+t_squared))
/home/emi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.


Exception: Test failed on one or more output criteria.
  Inputs:
    Residuals: [(44.82441747649942, 38.68401197990367), (42.88202882480793, 38.145177488199074), (44.86270187834573, 39.157618521797545), (44.17586354251248, 39.105100166860886), (44.1335431873508, 41.22959163490233), (45.24516380180746, 42.36450773929684), (47.10844698414792, 43.50648764715951), (43.805401980529815, 40.234640112017395), (42.41153275509475, 41.0965715884463), (42.3981584496721, 41.82019917700367), (45.28576586094998, 44.962982669238976), (43.463850186715995, 42.43424203051986), (45.99809806594046, 44.715546469338534), (46.33465697698835, 47.20106126258009), (42.86958322217579, 45.479611929239255)]
  Failures:
    Better learner did not exceed worse learner. Expected 10, found 4

# WARNING

I don't know whether the exception is correst or not. I won't spend more time testing. Maybe in the future I'll try it.